<h1 style="text-align: center; font-family: Verdana; font-size: 40px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #42288a; background-color: #ffffff;">DBSCAN + EfficientNet baseline</h1>

<h1 style="text-align: left; font-family: roboto; font-size: 20px; font-style: normal; font-weight: none; text-decoration: none; text-transform: none; font-variant: none;  color: #429ef5; background-color: #ffffff;">
    Density-Based Spacial Clustering of Applications with noise. It is an efficient clustering algoritm designed to be fast and effective. It is better than Kmeans clustering as it does not require us to select the number of clusters. <br>     <br>
    The Best case run time complexity of DBScan is <i> O(nlogn) </i> whereas for Kmeans, its <i> O(n<sup>2</sup>) </i> . It can even find arbitrary shaped clusters, for example a cluster that is surrounded by another cluster, but not connected to it. KMeans cannot do the same. <br>
    <br>
    In this notebook, we extract ImageNet EfficientNet embeddings and cluster them using <a href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html"> DBSCAN </a>.
    
</h1>


<br>
<h1 style="text-align: left; font-family: roboto; font-size: 25px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #c75d16; background-color: #ffffff;">
ACKNOWLEDGMENTS
</h1>

<h1 style="text-align: left; font-family: roboto; font-size: 20px; font-style: normal; font-weight: none; text-decoration: none; text-transform: none; font-variant: none;  color: #eb8f52; background-color: #ffffff;">
    Making this wouldn't have been possible without the help of following notebooks, do check them out!

</h1>

<ul>
    <li> <a href="https://www.kaggle.com/jollibobert/efficientnet-dbscan-baseline-cv-0-658"> https://www.kaggle.com/jollibobert/efficientnet-dbscan-baseline-cv-0-658 </a> </li>
    <li> <a href="https://www.kaggle.com/ammarnassanalhajali/cnn-with-keras-stater"> https://www.kaggle.com/ammarnassanalhajali/cnn-with-keras-stater </a> </li>
    <li> <a href="https://www.kaggle.com/debarshichanda/pytorch-arcface-gem-pooling-starter"> https://www.kaggle.com/debarshichanda/pytorch-arcface-gem-pooling-starter </a> </li>
</ul>
 


### LOADING LIBRARIES AND DATA

In [ ]:
import sys
sys.path.append("../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master")

# utils
import numpy as np 
import pandas as pd 
import math
import random 
import gc
import os 
from glob import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


# image utils
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# DBSCAN and sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from cuml.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from cuml.metrics.pairwise_distances import pairwise_distances
import cupy

from sklearn.model_selection import StratifiedKFold

# deep learning & models
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

In [ ]:
train = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")
test = pd.read_csv("../input/happy-whale-and-dolphin/sample_submission.csv")

train.head()

In [ ]:
train.shape

### SET CONFIG

In [ ]:
class Config:
    
    # models
    efficientnet_name = 'efficientnet-b2'  # 0 for b0, 1 for b1, etc
    
    # image size
    image_size = 256
    
    seed = 2022
    batch_size = 16
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    # dbscan metric - cosine/euclidean
    dbscan_metric = 'euclidean'
    

### DEFINING DATASET AND DATALOADER

<!-- <h1 style="text-align: left; font-family: arial; font-size: 15px; color: #91450a; background-color: #ffffff;">
    We will use a simple stratified train test split to tune the parameters of DBSCAN.
    
</h1> -->

In [ ]:
# transform for pytorch
transforms = A.Compose([
        A.Resize(Config.image_size, Config.image_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)


# Dataset class
class HappyWhaleDataset(Dataset):
    def __init__(self, df, transforms, test=False):
        self.df = df
        self.transforms = transforms
        self.test = test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        if self.test:            
            impath = f"../input/happy-whale-and-dolphin/test_images/{self.df.loc[idx, 'image']}"
        else:
            impath = f"../input/happy-whale-and-dolphin/train_images/{self.df.loc[idx, 'image']}"
            
        img = cv2.imread(impath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = self.transforms(image=img)["image"]
        
        return img


# map@5
# from - https://www.kaggle.com/pestipeti/explanation-of-map5-scoring-metric
def map_per_image(label, predictions): 
    try:
        return 1 / (predictions[:5].index(label) + 1)
    except ValueError:
        return 0.0

def map_per_set(labels, predictions):
    return np.mean([map_per_image(l, p) for l,p in zip(labels, predictions)])


In [ ]:
whale_dt = HappyWhaleDataset(train, transforms)
whale_dataloader = torch.utils.data.DataLoader(whale_dt, batch_size = Config.batch_size,
                                              num_workers = 2, pin_memory=True, shuffle=False)

whale_test_dt = HappyWhaleDataset(test, transforms, "test")
whale_test_dataloader = torch.utils.data.DataLoader(whale_test_dt, batch_size = Config.batch_size,
                                              num_workers = 2, pin_memory=True, shuffle=False)


len(whale_dataloader)

### DEFINING THE EFFICIENTNET

In [ ]:
class WhaleModel(nn.Module):
    def __init__(self):
        super(WhaleModel, self).__init__()
        
        self.model = EfficientNet.from_name(Config.efficientnet_name, include_top=False)
        self.model.load_state_dict(torch.load(glob(f"../input/efficientnet-pytorch/{Config.efficientnet_name}*")[0]), 
                                  strict=False)
        print(f"{Config.efficientnet_name} weights loaded")
                
    def forward(self, img):
        
        return self.model.forward(img)
        
model = WhaleModel()
model = model.to(Config.device)

### EXTRACT EMBEDDINGS

In [ ]:
# The embedding generation takes around 50 mins, I have generate the embeddings beforehand and uploaded it in a Dataset.
# You can uncomment the below code to generate embeddings for some other efficientnet.

# image_embs = np.load("../input/happywhaleeffnetembs/effb0-train.npy")

# Uncomment to generate embeddings
model.eval()

embeddings = []

for bind, img in enumerate(tqdm(whale_dataloader)):
    img = img.to(Config.device)
    embs = model(img)
    
    embeddings.append(embs.cpu().detach().numpy())    
    
    del img, embs
    gc.collect()

image_embs = np.concatenate(embeddings).squeeze()

del embeddings
gc.collect()

image_embs.shape

In [ ]:
# TEST IMAGES EMBEDDINGS

# test_image_embs = np.load("../input/happywhaleeffnetembs/effb0-test.npy")

# uncomment to generate embeddings manually
embeddings = []

for bind, img in enumerate(tqdm(whale_test_dataloader)):
    img = img.to(Config.device)
    embs = model(img)
    
    embeddings.append(embs.cpu().detach().numpy())    
    
    del img, embs
    gc.collect()

test_image_embs = np.concatenate(embeddings).squeeze()

del model, embeddings
gc.collect()

test_image_embs.shape

### FIT DBSCAN 

In [ ]:
def train_dbscan(eps, min_samples, image_embs, verbose=True, metric = Config.dbscan_metric):
    
    db = DBSCAN(eps = eps, 
                min_samples = min_samples, 
                metric = 'euclidean', 
                calc_core_sample_indices = True,
                verbose = 0)

    # trick to use cosine by normalizing euclidean
    # https://medium.com/ai-for-real/relationship-between-cosine-similarity-and-euclidean-distance-7e283a277dff
    if metric == 'cosine':
        norms = cupy.linalg.norm(cupy.array(image_embs), ord=2, axis=1)
        image_embs = image_embs/norms.get().reshape(-1, 1)
    
    db.fit(image_embs)
        
    if verbose:
        print(f"Generated {len(db.core_sample_indices_)} clusters from {image_embs.shape[0]} data points")
    
    return db, len(db.core_sample_indices_)

### CALCULATE TRAIN CV SCORE

<h1 style="text-align: left; font-family: arial; font-size: 15px; color: #91450a; background-color: #ffffff;">
    eps 
   
    It is the most important parameter in DBSCAN. It is the maximum distance between two samples for one to be considered as in the neighborhood of the other.
    
    We will tune this parameter, by maximizing validation mAP score. The total number of individual ids in train set must be the number of clusters in the train set. Let us sweep different values of eps to select the best one.
    
</h1>

In [ ]:
# trying out all values from 2 to 8 with steps of 0.1

numclus = []
for eps in tqdm(np.arange(2, 8, 0.1)):    
    numclus.append(train_dbscan(eps, 2, image_embs, verbose=False)[1])
    
    
plt.plot(np.arange(2, 8, 0.1), numclus)
plt.show()

In [ ]:
print(f"Train set has {len(train.individual_id.value_counts())} different ids")

<h1 style="text-align: left; font-family: arial; font-size: 17px; color: #000000; background-color: #ffffff;">
    
    From the graph we can see that the best eps should be 5.5 (approx)  , Let us train with this eps and check validation score
    
</h1>

In [ ]:
def dbscan_predict(db, train_embs, test_embs, metric = Config.dbscan_metric):
    
    core_samples_embs = train_embs[db.core_sample_indices_, :]
    print("Core image embeddings shape ", core_samples_embs.shape)
    
    predictions = []
    
    print(f"Predicting on {test_embs.shape[0]} test embeddings")
    for i in tqdm(range(test_embs.shape[0])):
        pred = []
        
        dists = pairwise_distances(core_samples_embs, test_embs[[i,], :], metric = metric)
        top5 = sorted(list(zip(dists, db.core_sample_indices_)))[:5]
        
        vals = [x[0] for x in top5]
        inds = [x[1] for x in top5]
        indids = [train.loc[x, 'individual_id'] for x in inds]
        
        if vals[0] > db.eps:
            pred.append(-1)
            pred.extend(indids[:4])
        else:
            pred.extend(indids)
            
        predictions.append(pred)
        
    print(f"Generated {len(predictions)} predictions for the test set")
    
    return predictions

In [ ]:
dbscan_model, _ = train_dbscan(eps = 5.5,
                            min_samples = 2,
                            image_embs = image_embs)
  

# takes too much time to check complete train set - increase this if you want to check for more
CHECK_SIZE = 2500

predictions = dbscan_predict(dbscan_model, image_embs, image_embs[:CHECK_SIZE])
score = map_per_set(train.individual_id.tolist()[:CHECK_SIZE], predictions)

print(f"CV score - ", score)

### TEST PREDICTION

In [ ]:
test

In [ ]:
predictions = dbscan_predict(dbscan_model, image_embs, test_image_embs)


def process_preds(pred):
    s = " "
    if pred[0] == -1:
        s += "new_individual "
    else:
        s += str(pred[0]) + " "

    s += " ".join([str(x) for x in pred[1:]])

    s.strip()

    return s

test.loc[:, 'predlist'] = predictions
test.loc[:, 'predictions'] = test['predlist'].apply(process_preds)

test[['image', 'predictions']].to_csv("submission.csv", index=False)
pd.read_csv("submission.csv").head()

<br>
<h1 style="text-align: left; font-family: roboto; font-size: 25px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #c75d16; background-color: #ffffff;">
POSSIBLE IMPROVEMENTS
</h1>

<ul>
    <li> We can use larger efficientnets to extract embeddings. </li>
    <li> We can train all the folds with DBSCAN and then combine the predictions  </li>
    <li> We can also train the efficients with ArcFace loss and then perform DBSCAN for better clusters </li>
</ul>

<h1 style="text-align: left; font-family: roboto; font-size: 20px; font-style: normal; font-weight: none; text-decoration: none; text-transform: none; font-variant: none;  color: #eb8f52; background-color: #ffffff;">
    Show your appreciation by a simple upvote! 😊

</h1>